
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

### 0. Imports

In [1]:
import pandas as pd
import asyncio
import aiohttp
from selenium import webdriver
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

## 1. Get data from Red Eléctrica Española - Asynchronous

In [5]:
def get_month_url(year,geo_id, endpoint):
    if not endpoint == "generacion/estructura-renovables":
        endpoint = "demanda/evolucion"

    year_url = f"https://apidatos.ree.es/es/datos/{endpoint}?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={geo_id}"

    return year_url

In [6]:
# Función para realizar la solicitud y procesar la respuesta
async def fetch_and_process(year,ccaa,cod_ccaa, endpoint,session):

    url = get_month_url(year,cod_ccaa,endpoint)
    
    async with session.get(url) as response:
        if response.status != 200:
            print(f"Error {response.status} en la URL: {url}")
            return pd.DataFrame() 
        try:
            data = await response.json()  
        except aiohttp.ContentTypeError:
            print(f"Error en la respuesta JSON para la URL: {url}")
            return pd.DataFrame() 

        values_dict_list = data['included'][0]['attributes']['values']
        
        headers = list(values_dict_list[0].keys())
        headers.extend(["ccaa","cod_ccaa"])

        year_month_df = pd.DataFrame(
            [tuple([*value_dict.values(),ccaa,cod_ccaa]) for value_dict in values_dict_list],
            columns=headers
        )

    return year_month_df

async def get_endpoint_data(endpoint, communities):


    async with aiohttp.ClientSession() as session:
        tasks = [fetch_and_process(year,ccaa,cod_ccaa, endpoint, session) for year in range(2019,2022) for ccaa, cod_ccaa in communities.items()]
        results = await asyncio.gather(*tasks)

    final_df = pd.concat(results, ignore_index=True)
    return final_df

In [7]:
cod_comunidades = {'Ceuta': 8744,
                'Melilla': 8745,
                'Andalucía': 4,
                'Aragón': 5,
                'Cantabria': 6,
                'Castilla - La Mancha': 7,
                'Castilla y León': 8,
                'Cataluña': 9,
                'País Vasco': 10,
                'Principado de Asturias': 11,
                'Comunidad de Madrid': 13,
                'Comunidad Foral de Navarra': 14,
                'Comunitat Valenciana': 15,
                'Extremadura': 16,
                'Galicia': 17,
                'Illes Balears': 8743,
                'Canarias': 8742,
                'Región de Murcia': 21,
                'La Rioja': 20}

In [8]:
demanda_evolucion_df = await get_endpoint_data("demanda/evolucion",cod_comunidades)
demanda_evolucion_df

,value,percentage,datetime,ccaa,cod_ccaa
0,17830.239,1,2019-01-01T00:00:00.000+01:00,Ceuta,8744
1,16019.527,1,2019-02-01T00:00:00.000+01:00,Ceuta,8744
2,16598.235,1,2019-03-01T00:00:00.000+01:00,Ceuta,8744
3,15221.094,1,2019-04-01T00:00:00.000+02:00,Ceuta,8744
4,16753.540,1,2019-05-01T00:00:00.000+02:00,Ceuta,8744
...,...,...,...,...,...
679,130850.035,1,2021-08-01T00:00:00.000+02:00,La Rioja,20
680,133323.629,1,2021-09-01T00:00:00.000+02:00,La Rioja,20
681,135918.940,1,2021-10-01T00:00:00.000+02:00,La Rioja,20
682,140846.250,1,2021-11-01T00:00:00.000+01:00,La Rioja,20


In [9]:
demanda_evolucion_df = await get_endpoint_data("generacion/estructura-renovables",cod_comunidades)
demanda_evolucion_df

Error 502 en la URL: https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8744
Error 502 en la URL: https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date=2020-01-01T00:00&end_date=2020-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8744
Error 502 en la URL: https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date=2021-01-01T00:00&end_date=2021-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8744


,value,percentage,datetime,ccaa,cod_ccaa
0,5.053,0.009113,2019-01-01T00:00:00.000+01:00,Melilla,8745
1,5.526,0.011159,2019-02-01T00:00:00.000+01:00,Melilla,8745
2,6.561,0.012039,2019-03-01T00:00:00.000+01:00,Melilla,8745
3,7.925,0.016925,2019-04-01T00:00:00.000+02:00,Melilla,8745
4,8.683,0.016244,2019-05-01T00:00:00.000+02:00,Melilla,8745
...,...,...,...,...,...
642,14016.445,0.168879,2021-08-01T00:00:00.000+02:00,La Rioja,20
643,6190.632,0.105550,2021-09-01T00:00:00.000+02:00,La Rioja,20
644,2813.361,0.038771,2021-10-01T00:00:00.000+02:00,La Rioja,20
645,5796.517,0.036000,2021-11-01T00:00:00.000+01:00,La Rioja,20


# 2. Datos INE

In [16]:
chrome_options = webdriver.ChromeOptions()

# establacemos las preferencias que queremos
prefs = {
    "download.default_directory": "C:\\Users\\Miguel\\Desktop\\Hackio\\Semana_5-ETL\\Dia_1\\Lab1\\data\\extracted",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
actions = ActionChains(driver)

### 2.1 Datos demográficos

In [17]:
driver.get("https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990")

In [18]:
driver.execute_script("window.scrollBy(0, 500);")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.LINK_TEXT, "Por provincia"))).click()

driver.execute_script("window.scrollBy(0, 700);")
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.LINK_TEXT, "Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"))).click()


In [19]:
for i in range(4):
    element_i = driver.find_element(By.ID,'cri1').find_elements(By.CLASS_NAME,"jP_0")[i]
    actions.key_down(Keys.CONTROL).click(element_i).key_up(Keys.CONTROL).perform()

for i in range(3):
    element_i = driver.find_element(By.ID,'cri2').find_elements(By.CLASS_NAME,"jP_0")[i]
    actions.key_down(Keys.CONTROL).click(element_i).key_up(Keys.CONTROL).perform()

for i in range(3):
    element_i = driver.find_element(By.ID,'cri3').find_elements(By.CLASS_NAME,"jP_0")[i]
    actions.key_down(Keys.CONTROL).click(element_i).key_up(Keys.CONTROL).perform()

driver.execute_script("window.scrollBy(0, 300);")

In [20]:
for year in range(2019,2023): # select years 2019 to 2021, deselecting 2022
    element = driver.find_element(By.XPATH, f"//option[text()='{year}']")
    element.click()

In [21]:
driver.find_element(By.ID, 'botonConsulSele').click()

In [22]:
driver.find_element(By.ID,"btnDescargaForm").click()

In [23]:
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/article/div/div/iframe')))
driver.switch_to.frame(iframe)
driver.find_element(By.XPATH, "//label[text()='CSV: separado por ;']").click()

### 2.2 Datos economicos

In [27]:
driver.get("https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429")

In [31]:
driver.execute_script("window.scrollBy(0, 500);")
element = driver.find_element(By.ID, "c_10429")
if element:
    element.click()
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[text()='P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo. ']"))).click()


In [32]:
for year in range(2019,2021): # select years 2019 to 2021, deselecting 2022
    element = driver.find_element(By.XPATH, f"//option[text()='{year}']")
    element.click()

driver.find_element(By.ID, 'botonConsulSele').click()
driver.find_element(By.ID,"btnDescargaForm").click()

iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/article/div/div/iframe')))
driver.switch_to.frame(iframe)
driver.find_element(By.XPATH, "//label[text()='CSV: separado por ;']").click()